In [4]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

**Gaussian function**

The Gaussian function is the form of:

$f(x) = a * exp(-\frac{(x - b)^2}{2c^2})$

For given real constants *a*, *b*, and non-zero *c*

**1D Gaussian Distribution**

The formula of a 1D Gaussian distribution is the following:


$g(x ; \mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}}exp(-\frac{1}{2}\frac{(x - \mu)^2}{\sigma^2})$

Where
- *g* = probability density dunction of a normally distributed random variable *x*.
- *a* = $\frac{1}{\sigma\sqrt{2\pi}}$
- *b* = $\mu$
- *c* = $\sigma$

Sources: [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_function), [Youtube](https://www.youtube.com/watch?v=eho8xH3E6mE&ab_channel=AlexanderIhler)

**1D Log-normal Distribution**

The 1D Log-normal PDF formula is the following:

$g(x ; \mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}}exp(-\frac{1}{2}\frac{(lnx - \mu)^2}{\sigma^2})$

Let's first rewrite *a* (or $\frac{1}{\sigma\sqrt{2\pi}}$) in Python:

The tfp.distribution Normal function is written as the following:

```none
pdf(x; mu, sigma) = exp(-0.5 (x - mu)**2 / sigma**2) / Z
Z = (2 pi sigma**2)**0.5
```


Therefore:


```none
log_pdf(x; mu, sigma)

= log(exp(-0.5 (x - mu)**2 / sigma**2) / Z) 

= log(exp(-0.5 (x - mu)**2 / sigma**2)) - log(Z) 

= -0.5 (x - mu)**2 / sigma**2 - log(Z)

= -0.5 (x - mu)**2 / sigma**2 - log((2*pi*sigma**2)**0.5)

= -0.5 (x - mu)**2 / sigma**2 - log((2*pi*sigma**2)**0.5)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi*sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi*sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + 0.5*log(sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + 0.5*2*log(sigma)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + log(sigma)

```

Given:

```
log_unnormalized = -0.5 (x - mu)**2 / sigma**2
log_normalization = 0.5*log(2*pi) + log(sigma)
```

Then:

log_pdf(x; mu, sigma) = log_unnormalized - log_normalization

In [5]:
def _log_prob(self, x, loc, scale):
    scale = tf.convert_to_tensor(scale)
    log_unnormalized = -0.5 * tf.math.squared_difference(
        x / scale, loc / scale)
    log_normalization = tf.constant(
        0.5 * np.log(2. * np.pi), dtype=self.dtype) + tf.math.log(scale)
    return log_unnormalized - log_normalization

 **log_unnormalized** is equivalent to:
 
$log(exp(-\frac{1}{2}\frac{(x - \mu)^2}{\sigma^2})) = -\frac{1}{2}\frac{(x - \mu)^2}{\sigma^2} = -\frac{(x - \mu)^2}{2}$

where log = ln

Should the log_unnormalized be that following instead? Need to include \sigma^2 in the denominator?

In [8]:
'''

log_unnormalized = -0.5 * tf.math.squared_difference(
    x / scale, loc / scale) / sigma**2
    
'''

'\nlog_unnormalized = -0.5 * tf.math.squared_difference(\n    x / scale, loc / scale)\n'

**log_normalization** is equivalent to:

$log(\frac{1}{\sigma\sqrt{2\pi}}) -\frac{log(2\pi) + log(\sigma)}{2}$

**1D Student-t Distribution**

The tfp.distribution StudentT function is written as the following:

**Multivariate Gaussian Distribution**|